In [11]:
import pandas as pd
import yfinance as yf
from datetime import timedelta
import pandas_ta as ta


In [12]:
tickers = ['TSLA', 'RIVN', 'AAPL','NVDA']
years = 5

In [13]:
def data_download_daily(tickers, years):
    today = pd.Timestamp.today()
    days_in_years = 365*years
    start_date = (today - timedelta(days=days_in_years)).strftime('%Y-%m-%d')
    end_date = today
    days_in_years = 365*years
    prices_df = pd.DataFrame()
    for ticker in tickers:
        data = yf.download(ticker, start = start_date, end= end_date, interval='1D')
        prices_df[ticker] = data['Adj Close']
    return prices_df


In [14]:
prices_df = data_download_daily(tickers=tickers, years=years)
prices_df

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,TSLA,RIVN,AAPL,NVDA
Date,,,,
2019-03-01 00:00:00-05:00,19.652666,NaN,42.110859,38.848690
2019-03-04 00:00:00-05:00,19.024000,NaN,42.322666,38.930630
2019-03-05 00:00:00-05:00,18.436001,NaN,42.245632,38.866074
2019-03-06 00:00:00-05:00,18.416000,NaN,42.002556,37.756107
2019-03-07 00:00:00-05:00,18.439333,NaN,41.516388,37.063313
...,...,...,...,...
2024-02-22 00:00:00-05:00,197.410004,11.45,184.369995,785.380005
2024-02-23 00:00:00-05:00,191.970001,10.07,182.520004,788.169983
2024-02-26 00:00:00-05:00,199.399994,10.70,181.160004,790.919983


In [15]:
def calculate_my_strategy(prices_df):
    for ticker in prices_df.columns:
        prices_df[ticker + '_upper_50'] = prices_df[ticker].rolling(window = 45).mean() + abs(prices_df[ticker].rolling(window = 90).mean())*0.5
        prices_df[ticker + '_lower_50'] = prices_df[ticker].rolling(window = 45).mean() - abs(prices_df[ticker].rolling(window = 90).mean())*0.5
        prices_df[ticker + '_rolling_50'] = prices_df[ticker].rolling(window = 50).mean()
        prices_df[ticker + '_upper_threshold'] = (prices_df[ticker].shift(1) > prices_df[ticker + '_upper_50']) & (prices_df[ticker] < prices_df[ticker + '_upper_50'])
        prices_df[ticker + '_lower_threshold'] = (prices_df[ticker].shift(1) < prices_df[ticker + '_lower_50']) & (prices_df[ticker] > prices_df[ticker + '_lower_50'])
        prices_df[ticker + '_over'] = prices_df[ticker] > prices_df[ticker + '_upper_50']
        prices_df[ticker + '_under'] = prices_df[ticker] < prices_df[ticker + '_lower_50']
    return prices_df


In [16]:
my_strat = calculate_my_strategy(prices_df=prices_df)
my_strat

,TSLA,RIVN,AAPL,NVDA,TSLA_upper_50,TSLA_lower_50,TSLA_rolling_50,TSLA_upper_threshold,TSLA_lower_threshold,TSLA_over,...,AAPL_lower_threshold,AAPL_over,AAPL_under,NVDA_upper_50,NVDA_lower_50,NVDA_rolling_50,NVDA_upper_threshold,NVDA_lower_threshold,NVDA_over,NVDA_under
Date,,,,,,,,,,,,,,,,,,,,,
2019-03-01 00:00:00-05:00,19.652666,NaN,42.110859,38.848690,NaN,NaN,NaN,False,False,False,...,False,False,False,NaN,NaN,NaN,False,False,False,False
2019-03-04 00:00:00-05:00,19.024000,NaN,42.322666,38.930630,NaN,NaN,NaN,False,False,False,...,False,False,False,NaN,NaN,NaN,False,False,False,False
2019-03-05 00:00:00-05:00,18.436001,NaN,42.245632,38.866074,NaN,NaN,NaN,False,False,False,...,False,False,False,NaN,NaN,NaN,False,False,False,False
2019-03-06 00:00:00-05:00,18.416000,NaN,42.002556,37.756107,NaN,NaN,NaN,False,False,False,...,False,False,False,NaN,NaN,NaN,False,False,False,False
2019-03-07 00:00:00-05:00,18.439333,NaN,41.516388,37.063313,NaN,NaN,NaN,False,False,False,...,False,False,False,NaN,NaN,NaN,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-02-22 00:00:00-05:00,197.410004,11.45,184.369995,785.380005,326.544556,103.800778,218.0672,False,False,False,...,False,False,False,860.029531,332.156690,584.406199,False,False,False,False
2024-02-23 00:00:00-05:00,191.970001,10.07,182.520004,788.169983,324.880167,102.793611,217.1118,False,False,False,...,False,False,False,868.269530,336.690024,590.844199,False,False,False,False
2024-02-26 00:00:00-05:00,199.399994,10.70,181.160004,790.919983,323.292389,101.811611,216.3596,False,False,False,...,False,False,False,876.655810,341.409520,597.131199,False,False,False,False


In [23]:
def calculate_donchian(prices_df):
    donchian_df = pd.DataFrame(index=prices_df.index)  # DataFrame to store Donchian channels
    for ticker in prices_df.columns:
            donchian_df[f"{ticker}_dcl"], donchian_df[f"{ticker}_dcm"], donchian_df[f"{ticker}_dcu"] = ta.donchian(high = prices_df[ticker], low = prices_df[ticker], lower_length = 40, upper_length = 50)

    return donchian_df

In [31]:
aapl = yf.download('AAPL', start = '2020-01-01', end= '2024-02-28', interval='1D')
aapl


[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2020-01-02 00:00:00-05:00,74.059998,75.150002,73.797501,75.087502,73.059433,135480400
2020-01-03 00:00:00-05:00,74.287498,75.144997,74.125000,74.357498,72.349129,146322800
2020-01-06 00:00:00-05:00,73.447502,74.989998,73.187500,74.949997,72.925621,118387200
2020-01-07 00:00:00-05:00,74.959999,75.224998,74.370003,74.597504,72.582657,108872000
2020-01-08 00:00:00-05:00,74.290001,76.110001,74.290001,75.797501,73.750259,132079200
...,...,...,...,...,...,...
2024-02-21 00:00:00-05:00,181.940002,182.889999,180.660004,182.320007,182.320007,41529700
2024-02-22 00:00:00-05:00,183.479996,184.960007,182.460007,184.369995,184.369995,52292200
2024-02-23 00:00:00-05:00,185.009995,185.039993,182.229996,182.520004,182.520004,45074500


In [34]:
aapl.rename(columns={'High':'aapl_high', 'Low':'aapl_low'}, inplace=True)
aapl

,Open,aapl_high,aapl_low,Close,Adj Close,Volume
Date,,,,,,
2020-01-02 00:00:00-05:00,74.059998,75.150002,73.797501,75.087502,73.059433,135480400
2020-01-03 00:00:00-05:00,74.287498,75.144997,74.125000,74.357498,72.349129,146322800
2020-01-06 00:00:00-05:00,73.447502,74.989998,73.187500,74.949997,72.925621,118387200
2020-01-07 00:00:00-05:00,74.959999,75.224998,74.370003,74.597504,72.582657,108872000
2020-01-08 00:00:00-05:00,74.290001,76.110001,74.290001,75.797501,73.750259,132079200
...,...,...,...,...,...,...
2024-02-21 00:00:00-05:00,181.940002,182.889999,180.660004,182.320007,182.320007,41529700
2024-02-22 00:00:00-05:00,183.479996,184.960007,182.460007,184.369995,184.369995,52292200
2024-02-23 00:00:00-05:00,185.009995,185.039993,182.229996,182.520004,182.520004,45074500


In [36]:
aapl[['dcl', 'dcm', 'dcu']] = aapl.ta.donchian(high = 'aapl_high',low = 'aapl_low', lower_length=40, upper_length=50)
aapl

,Open,aapl_high,aapl_low,Close,Adj Close,Volume,dcl,dcm,dcu
Date,,,,,,,,,
2020-01-02 00:00:00-05:00,74.059998,75.150002,73.797501,75.087502,73.059433,135480400,NaN,NaN,NaN
2020-01-03 00:00:00-05:00,74.287498,75.144997,74.125000,74.357498,72.349129,146322800,NaN,NaN,NaN
2020-01-06 00:00:00-05:00,73.447502,74.989998,73.187500,74.949997,72.925621,118387200,NaN,NaN,NaN
2020-01-07 00:00:00-05:00,74.959999,75.224998,74.370003,74.597504,72.582657,108872000,NaN,NaN,NaN
2020-01-08 00:00:00-05:00,74.290001,76.110001,74.290001,75.797501,73.750259,132079200,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
2024-02-21 00:00:00-05:00,181.940002,182.889999,180.660004,182.320007,182.320007,41529700,179.25,189.434998,199.619995
2024-02-22 00:00:00-05:00,183.479996,184.960007,182.460007,184.369995,184.369995,52292200,179.25,189.434998,199.619995
2024-02-23 00:00:00-05:00,185.009995,185.039993,182.229996,182.520004,182.520004,45074500,179.25,189.434998,199.619995


In [30]:
donchain = calculate_donchian(prices_df)
donchain

,TSLA_dcl,TSLA_dcm,TSLA_dcu,RIVN_dcl,RIVN_dcm,RIVN_dcu,AAPL_dcl,AAPL_dcm,AAPL_dcu,NVDA_dcl,...,NVDA_upper_threshold_dcu,NVDA_lower_threshold_dcl,NVDA_lower_threshold_dcm,NVDA_lower_threshold_dcu,NVDA_over_dcl,NVDA_over_dcm,NVDA_over_dcu,NVDA_under_dcl,NVDA_under_dcm,NVDA_under_dcu
Date,,,,,,,,,,,,,,,,,,,,,
2019-03-01 00:00:00-05:00,DCL_40_50,DCM_40_50,DCU_40_50,DCL_40_50,DCM_40_50,DCU_40_50,DCL_40_50,DCM_40_50,DCU_40_50,DCL_40_50,...,DCU_40_50,DCL_40_50,DCM_40_50,DCU_40_50,DCL_40_50,DCM_40_50,DCU_40_50,DCL_40_50,DCM_40_50,DCU_40_50
2019-03-04 00:00:00-05:00,DCL_40_50,DCM_40_50,DCU_40_50,DCL_40_50,DCM_40_50,DCU_40_50,DCL_40_50,DCM_40_50,DCU_40_50,DCL_40_50,...,DCU_40_50,DCL_40_50,DCM_40_50,DCU_40_50,DCL_40_50,DCM_40_50,DCU_40_50,DCL_40_50,DCM_40_50,DCU_40_50
2019-03-05 00:00:00-05:00,DCL_40_50,DCM_40_50,DCU_40_50,DCL_40_50,DCM_40_50,DCU_40_50,DCL_40_50,DCM_40_50,DCU_40_50,DCL_40_50,...,DCU_40_50,DCL_40_50,DCM_40_50,DCU_40_50,DCL_40_50,DCM_40_50,DCU_40_50,DCL_40_50,DCM_40_50,DCU_40_50
2019-03-06 00:00:00-05:00,DCL_40_50,DCM_40_50,DCU_40_50,DCL_40_50,DCM_40_50,DCU_40_50,DCL_40_50,DCM_40_50,DCU_40_50,DCL_40_50,...,DCU_40_50,DCL_40_50,DCM_40_50,DCU_40_50,DCL_40_50,DCM_40_50,DCU_40_50,DCL_40_50,DCM_40_50,DCU_40_50
2019-03-07 00:00:00-05:00,DCL_40_50,DCM_40_50,DCU_40_50,DCL_40_50,DCM_40_50,DCU_40_50,DCL_40_50,DCM_40_50,DCU_40_50,DCL_40_50,...,DCU_40_50,DCL_40_50,DCM_40_50,DCU_40_50,DCL_40_50,DCM_40_50,DCU_40_50,DCL_40_50,DCM_40_50,DCU_40_50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-02-22 00:00:00-05:00,DCL_40_50,DCM_40_50,DCU_40_50,DCL_40_50,DCM_40_50,DCU_40_50,DCL_40_50,DCM_40_50,DCU_40_50,DCL_40_50,...,DCU_40_50,DCL_40_50,DCM_40_50,DCU_40_50,DCL_40_50,DCM_40_50,DCU_40_50,DCL_40_50,DCM_40_50,DCU_40_50
2024-02-23 00:00:00-05:00,DCL_40_50,DCM_40_50,DCU_40_50,DCL_40_50,DCM_40_50,DCU_40_50,DCL_40_50,DCM_40_50,DCU_40_50,DCL_40_50,...,DCU_40_50,DCL_40_50,DCM_40_50,DCU_40_50,DCL_40_50,DCM_40_50,DCU_40_50,DCL_40_50,DCM_40_50,DCU_40_50
2024-02-26 00:00:00-05:00,DCL_40_50,DCM_40_50,DCU_40_50,DCL_40_50,DCM_40_50,DCU_40_50,DCL_40_50,DCM_40_50,DCU_40_50,DCL_40_50,...,DCU_40_50,DCL_40_50,DCM_40_50,DCU_40_50,DCL_40_50,DCM_40_50,DCU_40_50,DCL_40_50,DCM_40_50,DCU_40_50
